Sanity check to verify Gemma example from [Flax](https://github.com/google/flax/tree/main/examples/gemma) cloned into my repository works as intented.

In [ ]:
! pip install --no-deps -U flax
! pip install jaxtyping kagglehub treescope

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.8 MB/s eta 0:00:00


In [6]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
import os
from google.colab import userdata # `userdata` is a Colab API.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [7]:
from IPython.display import clear_output

VARIANT = '2b-it' # @param ['2b', '2b-it', '7b', '7b-it'] {type:"string"}
weights_dir = kagglehub.model_download(f'google/gemma/Flax/{VARIANT}')
ckpt_path = f'{weights_dir}/{VARIANT}'
vocab_path = f'{weights_dir}/tokenizer.model'

In [8]:
from flax import nnx
import sentencepiece as spm

In [13]:
import sys
import tempfile
with tempfile.TemporaryDirectory() as tmp:
  # Create a temporary directory and clone the `flax` repo.
  # Then, append the `examples/gemma` folder to the path for loading the `gemma` modules.
  ! git clone https://github.com/Megan-Andrews/GSoC-2025.git {tmp}
  # ! ls {tmp}/src/models/gemma
  sys.path.append(f"{tmp}/src/models/gemma")
  import params as params_lib
  import sampler as sampler_lib
  import transformer as transformer_lib
  sys.path.pop();

Cloning into '/tmp/tmpqpsf0mo_'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 0), reused 19 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 30.31 KiB | 5.05 MiB/s, done.


In [14]:
params = params_lib.load_and_format_params(ckpt_path)


In [15]:
vocab = spm.SentencePieceProcessor()
vocab.Load(vocab_path)

True

In [16]:
transformer = transformer_lib.Transformer.from_params(params)
nnx.display(transformer)

In [17]:
sampler = sampler_lib.Sampler(
    transformer=transformer,
    vocab=vocab,
)

In [18]:
input_batch = [
    "\n# Python program for implementation of Bubble Sort\n\ndef bubbleSort(arr):",
  ]

out_data = sampler(
    input_strings=input_batch,
    total_generation_steps=300,  # The number of steps performed when generating a response.
  )

for input_string, out_string in zip(input_batch, out_data.text):
  print(f"Prompt:\n{input_string}\nOutput:\n{out_string}")
  print()
  print(10*'#')

Prompt:

# Python program for implementation of Bubble Sort

def bubbleSort(arr):
Output:

    for i in range(len(arr)):
        for j in range(len(arr)-i-1):
            if arr[j] > arr[j+1]:
                swap(arr, j, j+1)
    return arr

def swap(arr, i, j):
    temp = arr[i]
    arr[i] = arr[j]
    arr[j] = temp

# Driver code
arr = [5, 2, 8, 3, 1, 9]
print(bubbleSort(arr))

```

**Explanation:**

1. **bubbleSort Function**:
    - It takes a list `arr` as input.
    - It uses two nested for loops to iterate through the list.
    - The outer loop `i` iterates from the beginning of the list to the end of the list.
    - The inner loop `j` iterates from the second element of the list to the end of the list minus `i` (excluding the element at position `j` itself).
    - Inside the nested loops, it compares the elements at positions `j` and `j+1` in the list.
    - If `arr[j]` is greater than `arr[j+1]`, it swaps the elements at positions `j` and `j+1` in the list.
    -

##########
